<a href="https://colab.research.google.com/github/JosephKJ/DL-Tutorial/blob/master/4_ConvNet_Digit_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import transforms


class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 256, kernel_size=3)
        self.mp1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(256, 512, kernel_size=3)
        self.mp2 = nn.MaxPool2d(kernel_size=4, stride=2)
        self.fc = nn.Linear(512 * 4 * 4, 10)

    def forward(self, x):
        x = self.mp1(F.leaky_relu(self.conv1(x)))
        x = self.mp2(F.leaky_relu(self.conv2(x)))
        x = x.view(-1, 512 * 4 * 4)
        x = self.fc(x)
        return x


# Variables
batch_size = 64
learning_rate = 0.001
epochs = 2
device = torch.device('cuda:0')

# Data
train_data = datasets.MNIST('./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST('./data', train=False, download=True, transform=transforms.ToTensor())
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)


# Helper Functions
def compute_accuracy(output, target, topk=(1,)):
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    result = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        result.append(correct_k.mul_(100.0 / batch_size))
    return result


class Metrics:
    def __init__(self):
        self.val = 0
        self.sum = 0
        self.count = 0
        self.avg = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val*n
        self.count += n
        self.avg = self.sum / self.count


model = Network().to(device)
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for e in range(epochs):
    accuracy_metric = Metrics()
    loss_metric = Metrics()
    for index, (x, y) in enumerate(train_dataloader):
        x = x.to(device)
        y = y.to(device)

        y_pred = model(x)

        loss = F.cross_entropy(y_pred, y)

        opt.zero_grad()
        loss.backward()
        opt.step()

        # Book-keeping
        accuracy = compute_accuracy(y_pred, y)[0].item()
        accuracy_metric.update(accuracy)
        loss_metric.update(loss.item())

        if index % 100 == 0:
            print('Epoch: ' + str(e) + '\t | \t Loss: ' + str(loss_metric.avg) + '\t | \tAccuracy: ' + str(accuracy_metric.avg))

# Test
model.eval()

accuracy_metric = Metrics()
for index, (x, y) in enumerate(test_dataloader):
    x = x.to(device)
    y = y.to(device)

    y_pred = model(x)
    accuracy = compute_accuracy(y_pred, y)[0].item()
    accuracy_metric.update(accuracy)

print('Test Accuracy: ' + str(accuracy_metric.avg))
